In [5]:
%%sh
## Dependencies
pip install simpletransformers
rm -fr apex
git clone https://github.com/NVIDIA/apex
cd apex
pip install -v --no-cache-dir ./
#pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

  Running setup.py bdist_wheel for seqeval: started
  Running setup.py bdist_wheel for seqeval: finished with status 'done'
  Stored in directory: /home/ec2-user/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
  Running setup.py bdist_wheel for sacremoses: started
  Running setup.py bdist_wheel for sacremoses: finished with status 'done'
  Stored in directory: /home/ec2-user/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built seqeval sacremoses
  Found existing installation: protobuf 3.5.2
    Uninstalling protobuf-3.5.2:
      Successfully uninstalled protobuf-3.5.2
Created temporary directory: /tmp/pip-ephem-wheel-cache-rv43ijf8
Created temporary directory: /tmp/pip-install-zysowutt
Processing /home/ec2-user/SageMaker/apex
  Created temporary directory: /tmp/pip-req-build-x7nf719q
  Running setup.py (path:/tmp/pip-req-build-x7nf719q/setup.py) egg_info for package from file:///home/ec2-user/SageMaker/apex
    R

fastai 1.0.60 requires nvidia-ml-py3, which is not installed.
transformers 2.8.0 has requirement tokenizers==0.5.2, but you'll have tokenizers 0.6.0 which is incompatible.
You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Cloning into 'apex'...
fastai 1.0.60 requires nvidia-ml-py3, which is not installed.
transformers 2.8.0 has requirement tokenizers==0.5.2, but you'll have tokenizers 0.6.0 which is incompatible.
You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import pandas as pd

In [16]:
df = pd.read_csv('data/train_preprocessed.csv')
df['labels'] = list(zip(df.toxic.tolist(), df.severe_toxic.tolist(), df.obscene.tolist(), df.threat.tolist(),  df.insult.tolist(), df.identity_hate.tolist()))
df['text'] = df['comment_text'].apply(lambda x: str(x).replace('\n', ' '))

df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,labels,text
0,0000997932d777bf,explanation why the edits made under my userna...,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",explanation why the edits made under my userna...
1,000103f0d9cfb60f,d'aww he matches this background colour i'm se...,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",d'aww he matches this background colour i'm se...
2,000113f07ec002fd,hey man i'm really not trying to edit war it's...,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",hey man i'm really not trying to edit war it's...
3,0001b41b1c6bb37e,more i can't make any real suggestions on impr...,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",more i can't make any real suggestions on impr...
4,0001d958c54c6e35,you sir are my hero any chance you remember wh...,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",you sir are my hero any chance you remember wh...


In [17]:
from sklearn.model_selection import train_test_split

train_df, eval_df = train_test_split(df, test_size=0.2)

In [18]:
from simpletransformers.classification import MultiLabelClassificationModel

model = MultiLabelClassificationModel('roberta', 'roberta-base', num_labels=6, args={'train_batch_size':2, 'gradient_accumulation_steps':16, 'learning_rate': 3e-5, 'num_train_epochs': 2, 'max_seq_length': 512})

In [19]:
!rm -fr outputs/

In [20]:
cpu_count=8
args = {
    "output_dir": "outputs/",
    "cache_dir": "cache_dir/",

    "fp16": True,
    "fp16_opt_level": "O1",
    "max_seq_length": 128,
    "train_batch_size": 8,
    "gradient_accumulation_steps": 1,
    "eval_batch_size": 8,
    "num_train_epochs": 1,
    "weight_decay": 0,
    "learning_rate": 4e-5,
    "adam_epsilon": 1e-8,
    "warmup_ratio": 0.06,
    "warmup_steps": 0,
    "max_grad_norm": 1.0,

    "logging_steps": 50,
    "save_steps": 2000,

    "overwrite_output_dir": False,
    "reprocess_input_data": False,
    "evaluate_during_training": False,

    "process_count": cpu_count - 2 if cpu_count > 2 else 1,
    "n_gpu": 1
}

In [21]:
model.train_model(train_df,overwrite_output_dir=True)


Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Running loss: 0.681203

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:113: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Running loss: 0.598679

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:224: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 0.009854Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 262144.0
Running loss: 0.005995Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 262144.0
Running loss: 0.003543Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 262144.0
Running loss: 0.403419Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 131072.0
Running loss: 0.002551Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 262144.0
Running loss: 0.164177Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 131072.0
Running loss: 0.490306Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 262144.0
Running loss: 0.077252Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 131072.0
Running loss: 0.151866Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 262144.0
Running loss: 0.801173Gradient overflow.  Skipping step, loss scaler 0 re

Running loss: 0.000483Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 524288.0
Running loss: 0.363993Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 524288.0
Running loss: 0.000488Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 524288.0
Running loss: 0.963787Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 262144.0
Running loss: 0.038708Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 262144.0
Running loss: 0.000340Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 262144.0
Running loss: 0.000440Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 131072.0
Running loss: 0.038579Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 65536.0
Running loss: 0.611042Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 262144.0
Running loss: 0.000849Gradient overflow.  Skipping step, loss scaler 0 red

In [24]:
test_df = pd.read_csv('data/test_preprocessed.csv')

to_predict = test_df.comment_text.apply(lambda x: str(x).replace('\n', ' ')).tolist()
preds, outputs = model.predict(to_predict)
sub_df = pd.DataFrame(outputs, columns=['toxic','severe_toxic','obscene','threat','insult','identity_hate'])

sub_df['id'] = test_df['id']
sub_df = sub_df[['id', 'toxic','severe_toxic','obscene','threat','insult','identity_hate']]

sub_df.to_csv('outputs/submission_roberta.csv', index=False)